This notebook runs the experiment for lookahead steps 100 to 500 for the large QVOL circuits. 

The goal is the have `lookahead` to beat the other passes in terms of depth, especially as the `lookahead_steps` increases. 

Since layout can heavily influence the depth of QVOL circuits, we will run these experiments with the most recent SabreLayout. 

In [1]:
from qiskit.transpiler.passes.routing.sabre_swap_v020_lookahead import SabreSwap
from qiskit.transpiler.passes.layout.sabre_layout_v020 import SabreLayout
from ulti.circuits import get_circuit_list, sort_circuits_by_depth
from ulti.coupling_maps import file_to_coupling_map
from ulti.pass_managers import generate_pass_managers, transpiled_data
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import random
random.seed(42)
import os

In [2]:
qc_name = "size5_depth_100-500"
cp_name = "hex_7"

version = "20" # version for the routing pass after _v0.
lp_name = "sabre_0.20"
num_shots = 5
rp = SabreSwap
lp = SabreLayout
cm = file_to_coupling_map(f'coupling_maps/{cp_name}.txt')
qc_list = get_circuit_list([f'circuits/{qc_name}/'])
qc_list = sort_circuits_by_depth(qc_list)

fast_layout = True

In [3]:
for lookahead in range(10, 101, 10):
    experiment_name = f"sabre_look_{lookahead}"

    pm = generate_pass_managers(num_shots, rp, lp, cm, lookahead=lookahead, fast_layout=fast_layout)
    rp_name = experiment_name + "_v0." + version

    csv_file_path = f'data/{qc_name}/{experiment_name}_v0{version}.csv'
    directory = f'data/{qc_name}'
    if not os.path.exists(directory):
            os.makedirs(directory)

    data_list = []
    for qc_idx, qc in enumerate(qc_list):
        data = transpiled_data(qc, pm)
        data['routing pass'] = rp_name
        data['layout pass'] = lp_name
        data['coupling map'] = cp_name
        data['circuit label'] = qc_idx
        data_list.append(data)

        print(f"Finished {qc_idx}")
    print("Finished all")

    df = pd.DataFrame(data_list)
    df.to_csv(csv_file_path, index=False)



Finished 0
Finished 1
